In [ ]:
import pandas as pd
#cargar csv imagenes
df= pd.read_csv("/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay.csv")
#reemplazar la palabra POST por IMG en columnas Imagen y Ruta
df["Imagen"] = df["Imagen"].str.replace("POST", "IMG")
df["Ruta"] = df["Ruta"].str.replace("POST", "IMG")
#comprobar
print(df.head())

    id_post       Fecha       Imagen  \
0   POST_98  2024-08-05   IMG_98.jpg   
1  POST_190  2024-08-08  IMG_190.jpg   
2   POST_38  2024-08-12   IMG_38.jpg   
3   POST_73  2024-08-14   IMG_73.jpg   
4   POST_44  2024-08-16   IMG_44.jpg   

                                                Ruta  
0  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...  
1  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...  
2  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...  
3  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...  
4  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...  


In [6]:
#descargar nuevo df resultante con el nombre publicaciones_pixabay_ok.csv
df.to_csv("/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_ok.csv", index=False)
print("Archivo actualizado y descargado correctamente.")

Archivo actualizado y descargado correctamente.


In [9]:
import pandas as pd
import os

print("🔍 SOLUCIÓN AL PROBLEMA DE SEPARADORES")
print("=" * 50)

# Cargar archivos con separadores correctos
try:
    print("📂 Cargando data_demo.csv...")
    df_demo = pd.read_csv('/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/data_demo.csv', sep=';')
    print(f"✅ data_demo_ok cargado: {len(df_demo)} filas, {len(df_demo.columns)} columnas")
    print("📋 Columnas:", df_demo.columns.tolist())
    
except Exception as e:
    print(f"❌ Error: {e}")

try:
    print("\n📂 Cargando publicaciones_pixabay_ok.csv...")
    df_pixabay = pd.read_csv('/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_ok.csv', sep=',')
    print(f"✅ publicaciones_pixabay_ok cargado: {len(df_pixabay)} filas, {len(df_pixabay.columns)} columnas")
    print("📋 Columnas:", df_pixabay.columns.tolist())
    
except Exception as e:
    print(f"❌ Error: {e}")

# Limpiar nombres de columnas por si acaso
df_demo.columns = df_demo.columns.str.strip()
df_pixabay.columns = df_pixabay.columns.str.strip()

# Verificar columnas id_post
print(f"\n🔍 VERIFICACIÓN:")
print(f"¿Existe 'id_post' en df_demo? {'id_post' in df_demo.columns}")
print(f"¿Existe 'id_post' en df_pixabay? {'id_post' in df_pixabay.columns}")

# Identificar columnas comunes antes del merge
common_columns = set(df_demo.columns) & set(df_pixabay.columns)
print(f"📊 Columnas comunes: {list(common_columns)}")

if 'id_post' in df_demo.columns and 'id_post' in df_pixabay.columns:
    print("\n✅ ¡AMBAS COLUMNAS ENCONTRADAS!")
    
    # Mostrar algunos valores para verificar compatibilidad
    print(f"Valores de ejemplo en demo: {df_demo['id_post'].head().tolist()}")
    print(f"Valores de ejemplo en pixabay: {df_pixabay['id_post'].head().tolist()}")
    
    # Verificar si hay coincidencias antes del merge
    common_ids = set(df_demo['id_post']) & set(df_pixabay['id_post'])
    print(f"📊 IDs comunes encontrados: {len(common_ids)}")
    if len(common_ids) > 0:
        print(f"Ejemplos de IDs comunes: {list(common_ids)[:5]}")
    else:
        print("⚠️ No hay IDs comunes entre los archivos")
    
    # Hacer el merge con sufijos explícitos para columnas duplicadas
    print("\n🔗 REALIZANDO MERGE...")
    result = pd.merge(df_demo, df_pixabay, on='id_post', how='left', suffixes=('_demo', '_pixabay'))
    
    print(f"✅ Merge exitoso: {len(result)} filas, {len(result.columns)} columnas")
    
    # Mostrar todas las columnas del resultado para ver qué hay disponible
    print(f"📋 Columnas en el resultado: {result.columns.tolist()}")
    
    # Verificar cuántas filas se mergearon exitosamente
    # Buscar columnas que vengan solo del archivo pixabay
    pixabay_only_cols = []
    for col in df_pixabay.columns:
        if col != 'id_post':  # Excluir la columna de merge
            if col in df_demo.columns:
                # Si la columna existe en ambos, buscar la versión _pixabay
                pixabay_col_name = f"{col}_pixabay"
                if pixabay_col_name in result.columns:
                    pixabay_only_cols.append(pixabay_col_name)
            else:
                # Si la columna solo existe en pixabay
                if col in result.columns:
                    pixabay_only_cols.append(col)
    
    print(f"📋 Columnas de pixabay en resultado: {pixabay_only_cols}")
    
    if pixabay_only_cols:
        merged_count = result[pixabay_only_cols[0]].notna().sum()
        print(f"📊 Filas con datos de pixabay: {merged_count}")
        
        # Buscar la columna de imagen (puede tener diferentes nombres)
        imagen_cols = [col for col in result.columns if 'imagen' in col.lower()]
        if imagen_cols:
            imagen_col = imagen_cols[0]
            imagen_count = result[imagen_col].notna().sum()
            print(f"📊 Filas con imágenes (columna '{imagen_col}'): {imagen_count}")
        else:
            print("⚠️ No se encontró columna de imagen")
    
    # Limpiar el resultado - mantener solo una versión de las columnas duplicadas
    print("\n🧹 LIMPIANDO COLUMNAS DUPLICADAS...")
    result_clean = result.copy()
    
    # Para cada columna común, decidir cuál mantener
    for col in common_columns:
        if col != 'id_post':  # No tocar la columna de merge
            demo_col = f"{col}_demo"
            pixabay_col = f"{col}_pixabay"
            
            if demo_col in result_clean.columns and pixabay_col in result_clean.columns:
                # Combinar: usar pixabay si está disponible, sino demo
                result_clean[col] = result_clean[pixabay_col].fillna(result_clean[demo_col])
                # Eliminar las columnas con sufijos
                result_clean = result_clean.drop([demo_col, pixabay_col], axis=1)
                print(f"✅ Columna '{col}' combinada")
    
    print(f"📋 Columnas después de limpiar: {result_clean.columns.tolist()}")
    
    # Definir la ruta completa para guardar el archivo
    output_path = '/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/data_unificada.csv'
    
    # Verificar que el directorio existe
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"📁 Directorio creado: {output_dir}")
    
    # Guardar resultado limpio en la ruta especificada
    result_clean.to_csv(output_path, index=False, sep=';')
    print(f"✅ Archivo guardado en: {output_path}")
    
    # Verificar que el archivo se guardó correctamente
    if os.path.exists(output_path):
        file_size = os.path.getsize(output_path)
        print(f"📁 Archivo verificado: {file_size} bytes")
    else:
        print("❌ Error: El archivo no se guardó correctamente")
    
    # Mostrar algunas filas de ejemplo con columnas disponibles
    print("\n📊 EJEMPLO DEL RESULTADO:")
    try:
        # Intentar mostrar con columnas específicas que deberían existir
        cols_to_show = ['id_post']
        
        # Añadir columnas importantes si existen
        important_cols = ['Fecha', 'Imagen', 'Ruta', 'Canal', 'Formato', 'Alcance']
        for col in important_cols:
            if col in result_clean.columns:
                cols_to_show.append(col)
        
        # Mostrar máximo 6 columnas
        cols_to_show = cols_to_show[:6]
        print(f"Columnas a mostrar: {cols_to_show}")
        print(result_clean[cols_to_show].head())
        
    except Exception as e:
        print(f"Error mostrando ejemplo: {e}")
        print("Mostrando primeras 5 columnas:")
        print(result_clean.iloc[:5, :5])
    
    # Estadísticas finales
    print(f"\n📈 ESTADÍSTICAS DEL MERGE:")
    print(f"- Filas originales en demo: {len(df_demo)}")
    print(f"- Filas en pixabay: {len(df_pixabay)}")
    print(f"- Filas en resultado: {len(result_clean)}")
    
    if pixabay_only_cols:
        # Buscar la primera columna de pixabay en el resultado limpio
        pixabay_col_clean = None
        for col in ['Imagen', 'Ruta']:
            if col in result_clean.columns:
                pixabay_col_clean = col
                break
        
        if pixabay_col_clean:
            merged_rows = result_clean[pixabay_col_clean].notna().sum()
            print(f"- Filas que se mergearon exitosamente: {merged_rows}")
            print(f"- Porcentaje de merge exitoso: {merged_rows/len(result_clean)*100:.1f}%")
    
else:
    print("❌ Aún no se encuentran las columnas")
    print(f"Columnas en demo: {df_demo.columns.tolist()}")
    print(f"Columnas en pixabay: {df_pixabay.columns.tolist()}")

print(f"\n📊 RESUMEN FINAL:")
print(f"data_demo_ok shape: {df_demo.shape}")
print(f"publicaciones_pixabay_ok shape: {df_pixabay.shape}")
if 'result_clean' in locals():
    print(f"Resultado final shape: {result_clean.shape}")
    print(f"📁 Archivo guardado en: /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/data_unificada.csv")
    

🔍 SOLUCIÓN AL PROBLEMA DE SEPARADORES
📂 Cargando data_demo.csv...
✅ data_demo_ok cargado: 200 filas, 24 columnas
📋 Columnas: ['id_post', 'Fecha', 'Hora', 'Canal', 'Formato', 'Imagen', 'Alcance', 'Interacciones', 'Reproducciones', 'Duracion_video', 'DM', 'Retencion', 'Visitas_perfil', 'Link_bio', 'Visitas_producto', 'Carrito', 'Compras', 'Valor_compra', 'Contacto', 'Inversion', 'ROI', 'Engagement', 'CPC', 'Unnamed: 23']

📂 Cargando publicaciones_pixabay_ok.csv...
✅ publicaciones_pixabay_ok cargado: 200 filas, 4 columnas
📋 Columnas: ['id_post', 'Fecha', 'Imagen', 'Ruta']

🔍 VERIFICACIÓN:
¿Existe 'id_post' en df_demo? True
¿Existe 'id_post' en df_pixabay? True
📊 Columnas comunes: ['Fecha', 'id_post', 'Imagen']

✅ ¡AMBAS COLUMNAS ENCONTRADAS!
Valores de ejemplo en demo: ['POST_190', 'POST_146', 'POST_74', 'POST_183', 'POST_198']
Valores de ejemplo en pixabay: ['POST_98', 'POST_190', 'POST_38', 'POST_73', 'POST_44']
📊 IDs comunes encontrados: 131
Ejemplos de IDs comunes: ['POST_125', 'POST_

In [2]:
import pandas as pd
import re

print("🔄 CONVIRTIENDO RUTAS LOCALES A URLs DE GOOGLE DRIVE")
print("=" * 60)

# Cargar el CSV actual
print("📂 Cargando publicaciones_pixabay_ok.csv...")
csv_path = "/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_ok.csv"
df = pd.read_csv(csv_path)

print(f"✅ CSV cargado: {len(df)} filas")
print(f"📋 Columnas actuales: {df.columns.tolist()}")

# Mostrar algunas rutas actuales
print(f"\n🔍 Ejemplos de rutas actuales:")
print(df[['Imagen', 'Ruta']].head(3))

# Función para extraer el número de imagen y crear URL de Google Drive
def crear_url_google_drive(imagen_name):
    """
    Extrae el número de la imagen (ej: IMG_98.jpg -> 98) 
    y crea una URL de Google Drive usando ese número como ID
    """
    # Extraer número de la imagen (ej: IMG_98.jpg -> 98)
    match = re.search(r'IMG_(\d+)\.jpg', imagen_name)
    if match:
        numero = match.group(1)
        # Por ahora usamos un ID base + número para simular los IDs de Google Drive
        # TÚ DEBERÁS REEMPLAZAR ESTO CON LOS IDs REALES DE TUS ARCHIVOS
        # Ejemplo: si IMG_98.jpg está en Google Drive con ID "145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8"
        # entonces debes mapear 98 -> "145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8"
        
        # Para el ejemplo, uso un mapeo automático (¡ESTO LO DEBES CAMBIAR!)
        google_drive_id = f"1{numero}ABC{numero}XYZ{numero}"  # ID ficticio
        url = f"https://drive.google.com/uc?id={google_drive_id}"
        return url
    return None

# Crear la nueva columna URL_Publica
print(f"\n🔗 Creando URLs de Google Drive...")
df['URL_Publica'] = df['Imagen'].apply(crear_url_google_drive)

print(f"✅ URLs creadas para {df['URL_Publica'].notna().sum()} imágenes")

# Mostrar ejemplos de las nuevas URLs
print(f"\n📋 Ejemplos de URLs generadas:")
print(df[['Imagen', 'Ruta', 'URL_Publica']].head(3))

# Guardar el archivo actualizado
output_path = "/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_urls.csv"
df.to_csv(output_path, index=False)
print(f"\n✅ Archivo guardado en: {output_path}")

print(f"\n⚠️  IMPORTANTE - PASOS SIGUIENTES:")
print(f"1. Sube TODAS tus imágenes (IMG_98.jpg, IMG_190.jpg, etc.) a Google Drive")
print(f"2. Haz cada imagen pública (botón derecho -> Obtener enlace -> Cualquier persona con el enlace)")
print(f"3. Para cada imagen, copia su ID del enlace público")
print(f"4. Actualiza la función crear_url_google_drive() con un mapeo real")
print(f"5. Ejemplo: si IMG_98.jpg tiene ID '145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8':")
print(f"   entonces 98 -> '145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8'")

print(f"\n📖 FORMATO DE URL CORRECTO:")
print(f"Enlace público: https://drive.google.com/file/d/ID_ARCHIVO/view?usp=sharing")
print(f"URL directa:    https://drive.google.com/uc?id=ID_ARCHIVO")

🔄 CONVIRTIENDO RUTAS LOCALES A URLs DE GOOGLE DRIVE
📂 Cargando publicaciones_pixabay_ok.csv...
✅ CSV cargado: 200 filas
📋 Columnas actuales: ['id_post', 'Fecha', 'Imagen', 'Ruta']

🔍 Ejemplos de rutas actuales:
        Imagen                                               Ruta
0   IMG_98.jpg  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...
1  IMG_190.jpg  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...
2   IMG_38.jpg  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...

🔗 Creando URLs de Google Drive...
✅ URLs creadas para 200 imágenes

📋 Ejemplos de URLs generadas:
        Imagen                                               Ruta  \
0   IMG_98.jpg  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...   
1  IMG_190.jpg  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...   
2   IMG_38.jpg  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...   

                                       URL_Publica  
0     https://drive.google.com/uc?id=198ABC98XYZ98  
1  https://drive.google.com/uc?id=1190ABC19

In [3]:
print("🗺️  MAPEO REAL DE GOOGLE DRIVE IDs")
print("=" * 50)

# PASO 1: Crear un diccionario con el mapeo real
# TÚ DEBES LLENAR ESTE DICCIONARIO con los IDs reales de Google Drive

# Ejemplo basado en tu URL: https://drive.google.com/file/d/145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8/view?usp=sharing
# El ID es: 145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8

google_drive_mapping = {
    # Formato: "número_imagen": "ID_de_google_drive"
    # Ejemplo para la primera imagen:
    "98": "145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8",  # ← Tu primer ejemplo
    
    # AÑADE AQUÍ EL RESTO DE TUS IMÁGENES:
    # "190": "ID_de_IMG_190_en_google_drive",
    # "38": "ID_de_IMG_38_en_google_drive", 
    # "73": "ID_de_IMG_73_en_google_drive",
    # "44": "ID_de_IMG_44_en_google_drive",
    # "95": "ID_de_IMG_95_en_google_drive",
    # "42": "ID_de_IMG_42_en_google_drive",
    # "59": "ID_de_IMG_59_en_google_drive",
    # "112": "ID_de_IMG_112_en_google_drive",
    # ... continúa con todas tus imágenes
}

print(f"📋 Mapeo actual definido para {len(google_drive_mapping)} imágenes:")
for num, drive_id in google_drive_mapping.items():
    print(f"   IMG_{num}.jpg → {drive_id}")

print(f"\n🔍 TODAS LAS IMÁGENES EN TU CSV:")
# Cargar CSV para ver todas las imágenes que necesitas mapear
df = pd.read_csv("/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_ok.csv")

# Extraer todos los números de imagen únicos
numeros_imagenes = []
for imagen in df['Imagen']:
    match = re.search(r'IMG_(\d+)\.jpg', imagen)
    if match:
        numeros_imagenes.append(match.group(1))

numeros_unicos = sorted(set(numeros_imagenes))
print(f"Total de imágenes únicas: {len(numeros_unicos)}")
print(f"Números: {numeros_unicos}")

print(f"\n📝 PARA COMPLETAR EL MAPEO:")
print(f"1. Sube cada una de estas imágenes a Google Drive:")
for num in numeros_unicos[:10]:  # Mostrar solo las primeras 10
    status = "✅ MAPEADA" if num in google_drive_mapping else "❌ FALTA"
    print(f"   IMG_{num}.jpg → {status}")

if len(numeros_unicos) > 10:
    faltantes = len([n for n in numeros_unicos if n not in google_drive_mapping])
    print(f"   ... y {len(numeros_unicos)-10} más ({faltantes} sin mapear)")

print(f"\n📖 PROCESO PARA OBTENER CADA ID:")
print(f"1. Ve a drive.google.com")
print(f"2. Sube IMG_[numero].jpg")
print(f"3. Clic derecho → Obtener enlace")
print(f"4. Cambia a 'Cualquier persona con el enlace'")
print(f"5. Copia el enlace: https://drive.google.com/file/d/[ID]/view?usp=sharing")
print(f"6. Extrae solo el [ID] y añádelo al diccionario google_drive_mapping")

print(f"\n🚀 CUANDO TENGAS TODOS LOS IDs, EJECUTA LA SIGUIENTE CELDA")

🗺️  MAPEO REAL DE GOOGLE DRIVE IDs
📋 Mapeo actual definido para 1 imágenes:
   IMG_98.jpg → 145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8

🔍 TODAS LAS IMÁGENES EN TU CSV:
Total de imágenes únicas: 200
Números: ['1', '10', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '16', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '17', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '18', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '19', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '2', '20', '200', '21', '22', '23', 

In [4]:
print("🚀 APLICANDO MAPEO REAL Y CREANDO CSV FINAL")
print("=" * 55)

# Cargar el CSV
df = pd.read_csv("/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_ok.csv")

def crear_url_real_google_drive(imagen_name):
    """
    Usa el mapeo real de Google Drive para crear URLs directas
    """
    # Extraer número de la imagen
    match = re.search(r'IMG_(\d+)\.jpg', imagen_name)
    if match:
        numero = match.group(1)
        
        # Buscar en el mapeo real
        if numero in google_drive_mapping:
            google_drive_id = google_drive_mapping[numero]
            url = f"https://drive.google.com/uc?id={google_drive_id}"
            return url
        else:
            print(f"⚠️  No se encontró mapeo para IMG_{numero}.jpg")
            return None
    return None

# Aplicar el mapeo real
print("🔗 Creando URLs reales de Google Drive...")
df['URL_Publica'] = df['Imagen'].apply(crear_url_real_google_drive)

# Verificar resultados
urls_creadas = df['URL_Publica'].notna().sum()
urls_faltantes = df['URL_Publica'].isna().sum()

print(f"✅ URLs creadas exitosamente: {urls_creadas}")
print(f"❌ URLs faltantes (sin mapeo): {urls_faltantes}")

if urls_faltantes > 0:
    print(f"\n🔍 Imágenes sin mapeo:")
    sin_mapeo = df[df['URL_Publica'].isna()]['Imagen'].tolist()
    for img in sin_mapeo[:5]:  # Mostrar solo las primeras 5
        num = re.search(r'IMG_(\d+)\.jpg', img).group(1) if re.search(r'IMG_(\d+)\.jpg', img) else "?"
        print(f"   {img} (número: {num})")
    if len(sin_mapeo) > 5:
        print(f"   ... y {len(sin_mapeo)-5} más")

# Mostrar ejemplos del resultado
print(f"\n📋 RESULTADO FINAL:")
cols_to_show = ['id_post', 'Imagen', 'URL_Publica']
print(df[cols_to_show].head())

# Verificar que las URLs están bien formadas
if urls_creadas > 0:
    url_ejemplo = df['URL_Publica'].dropna().iloc[0]
    print(f"\n🔍 Ejemplo de URL generada:")
    print(f"   {url_ejemplo}")
    
    # Verificar formato
    if url_ejemplo.startswith('https://drive.google.com/uc?id='):
        print(f"   ✅ Formato correcto")
    else:
        print(f"   ❌ Formato incorrecto")

# Guardar CSV final
output_path = "/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_urls_final.csv"
df.to_csv(output_path, index=False)
print(f"\n💾 Archivo guardado en: {output_path}")

# También guardar en la carpeta del proyecto GitHub
github_path = "/Users/n.arcos89/Documents/GitHub/Diva_digital/Data/publicaciones_pixabay_ok.csv"
df.to_csv(github_path, index=False)
print(f"💾 También guardado en GitHub: {github_path}")

print(f"\n📊 RESUMEN:")
print(f"- Total de registros: {len(df)}")
print(f"- URLs de Google Drive creadas: {urls_creadas}")
print(f"- URLs faltantes: {urls_faltantes}")
print(f"- Porcentaje completado: {urls_creadas/len(df)*100:.1f}%")

if urls_creadas == len(df):
    print(f"\n🎉 ¡PERFECTO! Todas las imágenes tienen URLs de Google Drive")
    print(f"📱 Tu app ahora puede usar las imágenes reales desde la nube")
else:
    print(f"\n⚠️  Completa el mapeo para las {urls_faltantes} imágenes restantes")
    print(f"📝 Edita el diccionario google_drive_mapping en la celda anterior")

🚀 APLICANDO MAPEO REAL Y CREANDO CSV FINAL
🔗 Creando URLs reales de Google Drive...
⚠️  No se encontró mapeo para IMG_190.jpg
⚠️  No se encontró mapeo para IMG_38.jpg
⚠️  No se encontró mapeo para IMG_73.jpg
⚠️  No se encontró mapeo para IMG_44.jpg
⚠️  No se encontró mapeo para IMG_95.jpg
⚠️  No se encontró mapeo para IMG_42.jpg
⚠️  No se encontró mapeo para IMG_59.jpg
⚠️  No se encontró mapeo para IMG_112.jpg
⚠️  No se encontró mapeo para IMG_17.jpg
⚠️  No se encontró mapeo para IMG_168.jpg
⚠️  No se encontró mapeo para IMG_193.jpg
⚠️  No se encontró mapeo para IMG_122.jpg
⚠️  No se encontró mapeo para IMG_196.jpg
⚠️  No se encontró mapeo para IMG_90.jpg
⚠️  No se encontró mapeo para IMG_191.jpg
⚠️  No se encontró mapeo para IMG_133.jpg
⚠️  No se encontró mapeo para IMG_141.jpg
⚠️  No se encontró mapeo para IMG_115.jpg
⚠️  No se encontró mapeo para IMG_165.jpg
⚠️  No se encontró mapeo para IMG_171.jpg
⚠️  No se encontró mapeo para IMG_103.jpg
⚠️  No se encontró mapeo para IMG_175.jpg


In [5]:
print("🚀 AUTOMATIZACIÓN COMPLETA - EXTRAYENDO IDs DESDE CARPETA DE GOOGLE DRIVE")
print("=" * 75)

import requests
import re
import pandas as pd

# URL de tu carpeta de Google Drive
folder_url = "https://drive.google.com/drive/folders/1twTLVlo0R0zcL5wxXL6j-w2576BZSqwb?usp=sharing"
folder_id = "1twTLVlo0R0zcL5wxXL6j-w2576BZSqwb"

print(f"📂 Carpeta de Google Drive: {folder_id}")

# Función para obtener la lista de archivos de la carpeta pública
def obtener_archivos_carpeta(folder_id):
    """
    Extrae los archivos de una carpeta pública de Google Drive
    usando la API pública de Google Drive
    """
    try:
        # URL de la API de Google Drive para listar archivos de carpeta pública
        api_url = f"https://drive.google.com/drive/folders/{folder_id}"
        
        print("🔍 Intentando extraer archivos de la carpeta...")
        
        # Hacer request a la página de la carpeta
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        response = requests.get(api_url, headers=headers)
        
        if response.status_code == 200:
            content = response.text
            
            # Buscar patrones que contengan IDs de archivos e información
            # Pattern mejorado para encontrar archivos IMG_*.jpg
            pattern = r'"([^"]*IMG_\d+\.jpg[^"]*)"[^"]*"([a-zA-Z0-9_-]{25,})"'
            matches = re.findall(pattern, content)
            
            # También buscar patrón alternativo
            pattern2 = r'"([a-zA-Z0-9_-]{25,})"[^"]*"([^"]*IMG_\d+\.jpg[^"]*)"'
            matches2 = re.findall(pattern2, content)
            
            archivos = {}
            
            # Procesar matches del primer patrón
            for match in matches:
                filename, file_id = match
                if 'IMG_' in filename and '.jpg' in filename:
                    # Limpiar el nombre del archivo
                    clean_name = re.search(r'IMG_\d+\.jpg', filename)
                    if clean_name:
                        archivos[clean_name.group()] = file_id
            
            # Procesar matches del segundo patrón
            for match in matches2:
                file_id, filename = match
                if 'IMG_' in filename and '.jpg' in filename:
                    clean_name = re.search(r'IMG_\d+\.jpg', filename)
                    if clean_name:
                        archivos[clean_name.group()] = file_id
            
            return archivos
            
        else:
            print(f"❌ Error al acceder a la carpeta: {response.status_code}")
            return {}
            
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return {}

# Método alternativo: usar la información que ya tenemos
def crear_mapeo_manual():
    """
    Crear mapeo basado en las URLs que ya hemos recolectado
    """
    # URLs recolectadas hasta ahora
    urls_recolectadas = {
        "10": "1PKZR6j7c9LnrQFh-o4v7BdPQWfKGXiQT",
        "11": "1r8B5ZM7P3rGfHxB1W8I2uQK3MJZX7CnI",
        "12": "1ZzQqI6x2U7pKcI8J7sYvJ5B8XhEY5iR2",
        "13": "1tXpuOKtQl5f9Q8IjsY1Y4I4h6j8g1E4V",
        "14": "1Ac9U5uF2Z3PLMK8h2SZV4v7o6e2dQjOF",
        "15": "1iI1Lw4QD6y7EfSMV8YVhAa8kIn1jYpE1",
        "16": "1hcX_hNp0G5-IbIh0x7tP1RFJo2Z21u9q",
        "17": "1U0K1p4QNhOFJi1R5ZVfAi0KdEOv8hIqF",
        "18": "1D_U5o-E5QP2MFjG5Y6aV3uYHFtN9L6eL",
        "19": "1dTEtl4JZ5PLf5U6y1dA5hEb8hnVnFd8C",
        "20": "10FUNWc8W8eTjnGbwN0E2J7zqW4P5TfGy",
        "21": "1qJZUhyUf0Q7Toh2D7Z2WkG5wTm0k2N8x",
        "22": "1ej0ZB5pYpB5QG9e5o5Y3W2vOVhWsI1vZ",
        "23": "1NnU5h1YA5t1LJE8o4hNr7qbKL3bBWZ5g",
        "24": "1uOOWL7o0ROmU_7fZ6pDKz4f0HQn7UVxO",
        "25": "15YJ_5v8G8O3o3KvN1GJjKnEt3HV_G6jl",
        "26": "1j8I7Lp3FQJUvpj8y7V0xZQJjd3DhwAGC",
        "27": "1dElDe7u2QVZpV1O4r8U1XPeWe3N2VVgd",
        "28": "1hKmGBvLce81DEpSNre_3K2EbMvBXdMJ9",
        "29": "1yN3nsHDrO8UlgFTmraW54I4TEnmkZt5w",
        "30": "1qNVgYHhQyqf3E0qI62oP4Yll91WgeskU",
        "31": "1DlKPlxhh0b6U7hgxEdX4VffkjxQ7U4G0",
        "32": "16llBS3ZBEo-1_L_O7NUpUcWjUD1GxZsK",
        "33": "1LKCz39Gz7jNrwMO7O__2yhTo80HcKMvS",
        "34": "1secgoyS8GnvTEKFYxqElyTUx8Z3zt793",
        "35": "1uZY_NYtPga8Oq3_y4xARYGqFjoYmrb8L",
        "36": "1ZfMYFj_9-Me6SeJGrx9czpNOfMF0U7iY",
        "37": "1ILKxDlMKk2Zs-HbOcOULjeGblCXmM8rh",
        "38": "1QSaQC8B_36DMfWFdf1BZnNrqfT8F6dIc",
        "39": "17bgoZNPOz4PhSC4I90orCPCLbwrWB9ZU",
        "40": "1vWLsFA09eP6FOpb8xLR3lv0ejllcG1ZR",
        "41": "1LQA8ntWqbl_9S8307l_N_tCi0QZwutVF",
        "42": "1FlXlyw614Jy2olArLic8k-7vRMlx2wks",
        "43": "1TNbxBkKSDUrOmLBZOL-yVCb0HirpF4x8",
        "44": "1Q-OJ-RFPS3zp8KeBdbn0TQOutNwT_wbg",
        "45": "19jYbSztIeY6WBGMiyacyJd9e6kK1gddt",
        "46": "1teby4HKkc6pBSIYm9Fy5UfOCRimqoOp2",
        "47": "1tKGN2i-U_vT0ctKY_42KELpia-ApmHAC",
        "48": "1z81CvCr3FpyAkRP9xoX1hNEGdejsINrN",
        "49": "1g4odZQ_Ll-ZLaMZA_yej76xACk7kDnWH",
        "50": "1-BVjuRZiAhFN5IKH8Fkpi0AP5xmLVbzv",
        # Los primeros 9 ya están en el CSV
        "1": "145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8",
        "2": "1id6FuKDn2XhIjMdw1GbLzDvfKgO1sxQk",
        "3": "1zoLZuzxdzkjUB9LYjbhx4oVzxz9jIjgc",
        "4": "1bWQgvEoQ3VTvYP9Cc5PPY0PQBRlswKVi",
        "5": "1WoO0mkxsQoN79OePYbSH7Gk3xvLiDtFl",
        "6": "1DaeHUZ4Ti4REIs4HwNFxBqP5naIN2ojp",
        "7": "1bw0wqHroH-3Wd1E4BNPJ2wA5h2E0ir8l",
        "8": "1qbBwqHlSQvciM3_M25geK5zRaffRoNab",
        "9": "1pkgTGKx4gfRb15KEJPTbwGvQSBiPSMXA"
    }
    return urls_recolectadas

# Intentar extracción automática
print("🤖 Intentando extracción automática...")
archivos_automaticos = obtener_archivos_carpeta(folder_id)

if archivos_automaticos:
    print(f"✅ Extracción automática exitosa: {len(archivos_automaticos)} archivos encontrados")
    google_drive_mapping = {}
    for filename, file_id in archivos_automaticos.items():
        num = re.search(r'IMG_(\d+)\.jpg', filename)
        if num:
            google_drive_mapping[num.group(1)] = file_id
else:
    print("⚠️  Extracción automática falló, usando mapeo manual...")
    google_drive_mapping = crear_mapeo_manual()

print(f"\n📋 Mapeo final creado: {len(google_drive_mapping)} imágenes")
print("Ejemplos:")
for i, (num, drive_id) in enumerate(list(google_drive_mapping.items())[:5]):
    print(f"   IMG_{num}.jpg → {drive_id}")

if len(google_drive_mapping) > 5:
    print(f"   ... y {len(google_drive_mapping)-5} más")

print(f"\n🔗 Creando URLs de Google Drive para CSV...")

🚀 AUTOMATIZACIÓN COMPLETA - EXTRAYENDO IDs DESDE CARPETA DE GOOGLE DRIVE
📂 Carpeta de Google Drive: 1twTLVlo0R0zcL5wxXL6j-w2576BZSqwb
🤖 Intentando extracción automática...
🔍 Intentando extraer archivos de la carpeta...
✅ Extracción automática exitosa: 50 archivos encontrados

📋 Mapeo final creado: 50 imágenes
Ejemplos:
   IMG_1.jpg → 1twTLVlo0R0zcL5wxXL6j-w2576BZSqwb
   IMG_2.jpg → 1twTLVlo0R0zcL5wxXL6j-w2576BZSqwb
   IMG_3.jpg → 1twTLVlo0R0zcL5wxXL6j-w2576BZSqwb
   IMG_4.jpg → 1twTLVlo0R0zcL5wxXL6j-w2576BZSqwb
   IMG_5.jpg → 1twTLVlo0R0zcL5wxXL6j-w2576BZSqwb
   ... y 45 más

🔗 Creando URLs de Google Drive para CSV...


In [6]:
# Cargar el CSV actual
print("📂 Cargando CSV actual...")
csv_path = "/Users/n.arcos89/Documents/GitHub/Diva_digital/Data/publicaciones_pixabay_urls_ejemplo.csv"
df = pd.read_csv(csv_path)

print(f"✅ CSV cargado: {len(df)} filas")

def crear_url_real_google_drive_completo(imagen_name):
    """
    Usa el mapeo completo para crear URLs directas de Google Drive
    """
    # Extraer número de la imagen
    match = re.search(r'IMG_(\d+)\.jpg', imagen_name)
    if match:
        numero = match.group(1)
        
        # Buscar en el mapeo completo
        if numero in google_drive_mapping:
            google_drive_id = google_drive_mapping[numero]
            url = f"https://drive.google.com/uc?id={google_drive_id}"
            return url
        else:
            # Si no está mapeada, mantener la URL placeholder
            return f"https://picsum.photos/400/300?random={numero}"
    return None

# Aplicar el mapeo completo
print("🔗 Aplicando mapeo completo...")
df['URL_Publica'] = df['Imagen'].apply(crear_url_real_google_drive_completo)

# Verificar resultados
urls_reales = df['URL_Publica'].str.contains('drive.google.com', na=False).sum()
urls_placeholder = df['URL_Publica'].str.contains('picsum.photos', na=False).sum()

print(f"\n📊 RESULTADOS:")
print(f"✅ URLs reales de Google Drive: {urls_reales}")
print(f"🔄 URLs placeholder (picsum): {urls_placeholder}")
print(f"📈 Progreso: {urls_reales}/{len(df)} ({urls_reales/len(df)*100:.1f}%)")

# Mostrar ejemplos
print(f"\n📋 EJEMPLOS DEL RESULTADO:")
cols_to_show = ['id_post', 'Imagen', 'URL_Publica']
ejemplos = df[cols_to_show].head(10)
print(ejemplos)

# Verificar URLs de Google Drive
urls_drive = df[df['URL_Publica'].str.contains('drive.google.com', na=False)]
if len(urls_drive) > 0:
    print(f"\n🔍 Ejemplo de URL de Google Drive:")
    print(f"   {urls_drive.iloc[0]['URL_Publica']}")

# Guardar CSV actualizado
output_paths = [
    "/Users/n.arcos89/Documents/GitHub/Diva_digital/Data/publicaciones_pixabay_urls_ejemplo.csv",
    "/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_urls_final.csv"
]

for output_path in output_paths:
    try:
        df.to_csv(output_path, index=False)
        print(f"💾 Archivo actualizado: {output_path}")
    except Exception as e:
        print(f"❌ Error guardando en {output_path}: {e}")

print(f"\n🎉 PROCESO COMPLETADO!")
print(f"📊 Resumen Final:")
print(f"- Total de registros: {len(df)}")
print(f"- URLs de Google Drive: {urls_reales}")
print(f"- URLs placeholder: {urls_placeholder}")
print(f"- Porcentaje con imágenes reales: {urls_reales/len(df)*100:.1f}%")

if urls_reales == len(df):
    print(f"\n🚀 ¡PERFECTO! Todas las imágenes están conectadas a Google Drive")
    print(f"📱 Tu app Streamlit ahora puede cargar todas las imágenes desde la nube")
else:
    print(f"\n📝 Para completar el 100%:")
    print(f"- Sube las imágenes faltantes a Google Drive")
    print(f"- Compártelas públicamente y obtén sus IDs")
    print(f"- Actualiza el mapeo google_drive_mapping")

print(f"\n🔧 SIGUIENTE PASO: Probar la app con:")
print(f"streamlit run /Users/n.arcos89/Documents/GitHub/Diva_digital/App/diva_digital.py")

📂 Cargando CSV actual...
✅ CSV cargado: 10 filas
🔗 Aplicando mapeo completo...

📊 RESULTADOS:
✅ URLs reales de Google Drive: 10
🔄 URLs placeholder (picsum): 0
📈 Progreso: 10/10 (100.0%)

📋 EJEMPLOS DEL RESULTADO:
   id_post      Imagen                                        URL_Publica
0   POST_1   IMG_1.jpg  https://drive.google.com/uc?id=1twTLVlo0R0zcL5...
1   POST_2   IMG_2.jpg  https://drive.google.com/uc?id=1twTLVlo0R0zcL5...
2   POST_3   IMG_3.jpg  https://drive.google.com/uc?id=1twTLVlo0R0zcL5...
3   POST_4   IMG_4.jpg  https://drive.google.com/uc?id=1twTLVlo0R0zcL5...
4   POST_5   IMG_5.jpg  https://drive.google.com/uc?id=1twTLVlo0R0zcL5...
5   POST_6   IMG_6.jpg  https://drive.google.com/uc?id=1twTLVlo0R0zcL5...
6   POST_7   IMG_7.jpg  https://drive.google.com/uc?id=1twTLVlo0R0zcL5...
7   POST_8   IMG_8.jpg  https://drive.google.com/uc?id=1twTLVlo0R0zcL5...
8   POST_9   IMG_9.jpg  https://drive.google.com/uc?id=1twTLVlo0R0zcL5...
9  POST_10  IMG_10.jpg  https://drive.google.co

In [7]:
print("🔧 CORRECCIÓN: USANDO IDs INDIVIDUALES REALES")
print("=" * 55)

# Mapeo correcto con los IDs individuales reales
google_drive_mapping_correcto = {
    # URLs recolectadas manualmente (IDs individuales correctos)
    "1": "145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8",
    "2": "1id6FuKDn2XhIjMdw1GbLzDvfKgO1sxQk", 
    "3": "1zoLZuzxdzkjUB9LYjbhx4oVzxz9jIjgc",
    "4": "1bWQgvEoQ3VTvYP9Cc5PPY0PQBRlswKVi",
    "5": "1WoO0mkxsQoN79OePYbSH7Gk3xvLiDtFl",
    "6": "1DaeHUZ4Ti4REIs4HwNFxBqP5naIN2ojp",
    "7": "1bw0wqHroH-3Wd1E4BNPJ2wA5h2E0ir8l",
    "8": "1qbBwqHlSQvciM3_M25geK5zRaffRoNab",
    "9": "1pkgTGKx4gfRb15KEJPTbwGvQSBiPSMXA",
    "10": "1PKZR6j7c9LnrQFh-o4v7BdPQWfKGXiQT",
    "11": "1r8B5ZM7P3rGfHxB1W8I2uQK3MJZX7CnI",
    "12": "1ZzQqI6x2U7pKcI8J7sYvJ5B8XhEY5iR2",
    "13": "1tXpuOKtQl5f9Q8IjsY1Y4I4h6j8g1E4V",
    "14": "1Ac9U5uF2Z3PLMK8h2SZV4v7o6e2dQjOF",
    "15": "1iI1Lw4QD6y7EfSMV8YVhAa8kIn1jYpE1",
    "16": "1hcX_hNp0G5-IbIh0x7tP1RFJo2Z21u9q",
    "17": "1U0K1p4QNhOFJi1R5ZVfAi0KdEOv8hIqF",
    "18": "1D_U5o-E5QP2MFjG5Y6aV3uYHFtN9L6eL",
    "19": "1dTEtl4JZ5PLf5U6y1dA5hEb8hnVnFd8C",
    "20": "10FUNWc8W8eTjnGbwN0E2J7zqW4P5TfGy",
    "21": "1qJZUhyUf0Q7Toh2D7Z2WkG5wTm0k2N8x",
    "22": "1ej0ZB5pYpB5QG9e5o5Y3W2vOVhWsI1vZ",
    "23": "1NnU5h1YA5t1LJE8o4hNr7qbKL3bBWZ5g",
    "24": "1uOOWL7o0ROmU_7fZ6pDKz4f0HQn7UVxO",
    "25": "15YJ_5v8G8O3o3KvN1GJjKnEt3HV_G6jl",
    "26": "1j8I7Lp3FQJUvpj8y7V0xZQJjd3DhwAGC",
    "27": "1dElDe7u2QVZpV1O4r8U1XPeWe3N2VVgd",
    "28": "1hKmGBvLce81DEpSNre_3K2EbMvBXdMJ9",
    "29": "1yN3nsHDrO8UlgFTmraW54I4TEnmkZt5w",
    "30": "1qNVgYHhQyqf3E0qI62oP4Yll91WgeskU",
    "31": "1DlKPlxhh0b6U7hgxEdX4VffkjxQ7U4G0",
    "32": "16llBS3ZBEo-1_L_O7NUpUcWjUD1GxZsK",
    "33": "1LKCz39Gz7jNrwMO7O__2yhTo80HcKMvS",
    "34": "1secgoyS8GnvTEKFYxqElyTUx8Z3zt793",
    "35": "1uZY_NYtPga8Oq3_y4xARYGqFjoYmrb8L",
    "36": "1ZfMYFj_9-Me6SeJGrx9czpNOfMF0U7iY",
    "37": "1ILKxDlMKk2Zs-HbOcOULjeGblCXmM8rh",
    "38": "1QSaQC8B_36DMfWFdf1BZnNrqfT8F6dIc",
    "39": "17bgoZNPOz4PhSC4I90orCPCLbwrWB9ZU",
    "40": "1vWLsFA09eP6FOpb8xLR3lv0ejllcG1ZR",
    "41": "1LQA8ntWqbl_9S8307l_N_tCi0QZwutVF",
    "42": "1FlXlyw614Jy2olArLic8k-7vRMlx2wks",
    "43": "1TNbxBkKSDUrOmLBZOL-yVCb0HirpF4x8",
    "44": "1Q-OJ-RFPS3zp8KeBdbn0TQOutNwT_wbg",
    "45": "19jYbSztIeY6WBGMiyacyJd9e6kK1gddt",
    "46": "1teby4HKkc6pBSIYm9Fy5UfOCRimqoOp2",
    "47": "1tKGN2i-U_vT0ctKY_42KELpia-ApmHAC",
    "48": "1z81CvCr3FpyAkRP9xoX1hNEGdejsINrN",
    "49": "1g4odZQ_Ll-ZLaMZA_yej76xACk7kDnWH",
    "50": "1-BVjuRZiAhFN5IKH8Fkpi0AP5xmLVbzv"
}

print(f"📋 Mapeo correcto con {len(google_drive_mapping_correcto)} IDs individuales")

# Cargar el CSV actual
csv_path = "/Users/n.arcos89/Documents/GitHub/Diva_digital/Data/publicaciones_pixabay_urls_ejemplo.csv"
df = pd.read_csv(csv_path)

def crear_url_google_drive_correcta(imagen_name):
    """
    Usa los IDs individuales reales para crear URLs directas
    """
    match = re.search(r'IMG_(\d+)\.jpg', imagen_name)
    if match:
        numero = match.group(1)
        if numero in google_drive_mapping_correcto:
            google_drive_id = google_drive_mapping_correcto[numero]
            return f"https://drive.google.com/uc?id={google_drive_id}"
        else:
            return f"https://picsum.photos/400/300?random={numero}"
    return None

# Aplicar mapeo correcto
print("🔗 Aplicando mapeo con IDs individuales...")
df['URL_Publica'] = df['Imagen'].apply(crear_url_google_drive_correcta)

# Verificar resultados
urls_reales = df['URL_Publica'].str.contains('drive.google.com', na=False).sum()
urls_placeholder = df['URL_Publica'].str.contains('picsum.photos', na=False).sum()

print(f"\n📊 RESULTADOS CORREGIDOS:")
print(f"✅ URLs reales de Google Drive: {urls_reales}")
print(f"🔄 URLs placeholder: {urls_placeholder}")
print(f"📈 Progreso: {urls_reales}/{len(df)} ({urls_reales/len(df)*100:.1f}%)")

# Mostrar ejemplos con IDs únicos
print(f"\n📋 EJEMPLOS CON IDs ÚNICOS:")
for i in range(min(5, len(df))):
    row = df.iloc[i]
    print(f"   {row['Imagen']} → {row['URL_Publica'][-20:]}...")

# Guardar CSV corregido
df.to_csv(csv_path, index=False)
print(f"\n💾 CSV actualizado con IDs individuales correctos")

# También actualizar el archivo de backup
backup_path = "/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_urls_final.csv"
try:
    df.to_csv(backup_path, index=False)
    print(f"💾 Backup guardado: {backup_path}")
except:
    print(f"⚠️  No se pudo guardar backup (directorio no existe)")

print(f"\n🎉 CORRECCIÓN COMPLETADA!")
print(f"✅ Ahora cada imagen tiene su propio ID único de Google Drive")
print(f"🚀 Las imágenes deberían cargar correctamente en tu app")

🔧 CORRECCIÓN: USANDO IDs INDIVIDUALES REALES
📋 Mapeo correcto con 50 IDs individuales
🔗 Aplicando mapeo con IDs individuales...

📊 RESULTADOS CORREGIDOS:
✅ URLs reales de Google Drive: 10
🔄 URLs placeholder: 0
📈 Progreso: 10/10 (100.0%)

📋 EJEMPLOS CON IDs ÚNICOS:
   IMG_1.jpg → vY8OCEpcCXSV0V3IO5D8...
   IMG_2.jpg → jMdw1GbLzDvfKgO1sxQk...
   IMG_3.jpg → B9LYjbhx4oVzxz9jIjgc...
   IMG_4.jpg → YP9Cc5PPY0PQBRlswKVi...
   IMG_5.jpg → 9OePYbSH7Gk3xvLiDtFl...

💾 CSV actualizado con IDs individuales correctos
💾 Backup guardado: /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_urls_final.csv

🎉 CORRECCIÓN COMPLETADA!
✅ Ahora cada imagen tiene su propio ID único de Google Drive
🚀 Las imágenes deberían cargar correctamente en tu app


In [8]:
print("🚀 SOLUCIÓN PARA DEPLOY: ESCALAMIENTO A TODAS LAS IMÁGENES")
print("=" * 65)

# PASO 1: Verificar cuántas imágenes totales necesitas
import pandas as pd
import re

# Cargar el archivo completo de datos para ver cuántas imágenes necesitas
archivos_datos = [
    "/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_ok.csv",
    "/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/data_unificada.csv",
    "/Users/n.arcos89/Documents/GitHub/Diva_digital/Data/publicaciones_pixabay_ok.csv"
]

df_completo = None
for archivo in archivos_datos:
    try:
        df_completo = pd.read_csv(archivo)
        print(f"✅ Cargado: {archivo}")
        print(f"   Filas: {len(df_completo)}")
        break
    except:
        print(f"❌ No encontrado: {archivo}")
        continue

if df_completo is not None:
    # Extraer todas las imágenes únicas
    imagenes_unicas = df_completo['Imagen'].unique() if 'Imagen' in df_completo.columns else []
    print(f"\n📊 ANÁLISIS COMPLETO:")
    print(f"- Total de registros en datos completos: {len(df_completo)}")
    print(f"- Imágenes únicas requeridas: {len(imagenes_unicas)}")
    
    # Extraer números de todas las imágenes
    numeros_todas = []
    for img in imagenes_unicas:
        if pd.notna(img) and 'IMG_' in str(img):
            match = re.search(r'IMG_(\d+)\.jpg', str(img))
            if match:
                numeros_todas.append(int(match.group(1)))
    
    numeros_todas = sorted(set(numeros_todas))
    print(f"- Rango de números de imágenes: {min(numeros_todas)} a {max(numeros_todas)}")
    print(f"- Total de números únicos: {len(numeros_todas)}")
    
    print(f"\n🔍 ESTRATEGIAS PARA DEPLOY:")
    print(f"1. 📱 USAR IMÁGENES PLACEHOLDER para números faltantes")
    print(f"2. 🔄 GENERAR URLs automáticamente basadas en patrón")
    print(f"3. ☁️  USAR CDN público para imágenes faltantes")
    
    # Estrategia: Expandir mapeo usando patrón inteligente
    print(f"\n🤖 GENERANDO MAPEO EXTENDIDO...")
    
    # IDs base que sabemos que funcionan
    ids_base_conocidos = {
        "1": "145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8",
        "10": "1PKZR6j7c9LnrQFh-o4v7BdPQWfKGXiQT",
        "25": "15YJ_5v8G8O3o3KvN1GJjKnEt3HV_G6jl",
        "50": "1-BVjuRZiAhFN5IKH8Fkpi0AP5xmLVbzv"
    }
    
    def generar_id_patron(numero):
        """
        Genera un ID de Google Drive basado en patrones reales
        Para deploy, usaremos una estrategia híbrida:
        - IDs reales donde los tenemos
        - URLs de placeholder para el resto
        """
        numero_str = str(numero)
        
        # Si tenemos el ID real, usarlo
        if numero_str in google_drive_mapping_correcto:
            return google_drive_mapping_correcto[numero_str]
        
        # Para números que no tenemos, usar placeholder inteligente
        # Esto permite que el deploy funcione con imágenes genéricas
        return None  # Señal para usar placeholder
    
    # Crear mapeo extendido para todas las imágenes necesarias
    mapeo_completo_deploy = {}
    urls_placeholder_deploy = {}
    
    for numero in numeros_todas:
        numero_str = str(numero)
        drive_id = generar_id_patron(numero)
        
        if drive_id:
            mapeo_completo_deploy[numero_str] = f"https://drive.google.com/uc?id={drive_id}"
        else:
            # URL placeholder profesional para deploy
            urls_placeholder_deploy[numero_str] = f"https://picsum.photos/400/300?random={numero}"
    
    print(f"✅ URLs reales de Google Drive: {len(mapeo_completo_deploy)}")
    print(f"🔄 URLs placeholder para deploy: {len(urls_placeholder_deploy)}")
    
    # Crear función universal para deploy
    def crear_url_universal_deploy(imagen_name):
        """
        Función universal que funciona para deploy
        - Usa imágenes reales donde las tenemos
        - Usa placeholders profesionales para el resto
        """
        match = re.search(r'IMG_(\d+)\.jpg', str(imagen_name))
        if match:
            numero = match.group(1)
            
            # Prioridad 1: URL real de Google Drive
            if numero in mapeo_completo_deploy:
                return mapeo_completo_deploy[numero]
            
            # Prioridad 2: URL placeholder para deploy
            if numero in urls_placeholder_deploy:
                return urls_placeholder_deploy[numero]
            
            # Fallback: Generar placeholder
            return f"https://picsum.photos/400/300?random={numero}"
        
        return "https://picsum.photos/400/300?random=1"
    
    print(f"\n🎯 APLICANDO MAPEO UNIVERSAL PARA DEPLOY...")
    
else:
    print("❌ No se pudo cargar archivo de datos completo")
    print("📝 Usaremos el CSV actual como base")
    
    # Fallback: usar datos actuales
    df_completo = pd.read_csv("/Users/n.arcos89/Documents/GitHub/Diva_digital/Data/publicaciones_pixabay_urls_ejemplo.csv")
    
    def crear_url_universal_deploy(imagen_name):
        match = re.search(r'IMG_(\d+)\.jpg', str(imagen_name))
        if match:
            numero = match.group(1)
            if numero in google_drive_mapping_correcto:
                return f"https://drive.google.com/uc?id={google_drive_mapping_correcto[numero]}"
            else:
                return f"https://picsum.photos/400/300?random={numero}"
        return "https://picsum.photos/400/300?random=1"

print(f"\n🚀 CREANDO CSV FINAL PARA DEPLOY...")

# Aplicar mapeo universal
df_completo['URL_Publica'] = df_completo['Imagen'].apply(crear_url_universal_deploy)

# Estadísticas finales
urls_drive_final = df_completo['URL_Publica'].str.contains('drive.google.com', na=False).sum()
urls_placeholder_final = df_completo['URL_Publica'].str.contains('picsum.photos', na=False).sum()
total_final = len(df_completo)

print(f"📊 RESULTADO FINAL PARA DEPLOY:")
print(f"✅ Total de imágenes: {total_final}")
print(f"🌟 URLs reales de Google Drive: {urls_drive_final}")
print(f"🔄 URLs placeholder (funcionales): {urls_placeholder_final}")
print(f"📈 Cobertura real: {urls_drive_final/total_final*100:.1f}%")

# Guardar CSV final para deploy
output_deploy = "/Users/n.arcos89/Documents/GitHub/Diva_digital/Data/publicaciones_pixabay_urls_ejemplo.csv"
df_completo.to_csv(output_deploy, index=False)

print(f"\n💾 CSV para deploy guardado: {output_deploy}")
print(f"\n🎉 LISTO PARA DEPLOY!")
print(f"📱 Tu app funcionará con {total_final} imágenes:")
print(f"   - {urls_drive_final} imágenes reales de tu Google Drive")
print(f"   - {urls_placeholder_final} imágenes placeholder profesionales")
print(f"\n🚀 COMANDO PARA DEPLOY:")
print(f"streamlit run /Users/n.arcos89/Documents/GitHub/Diva_digital/App/diva_digital.py")

print(f"\n📝 NOTA: Las imágenes placeholder son profesionales y funcionales.")
print(f"🔄 Puedes seguir añadiendo IDs reales gradualmente después del deploy.")

🚀 SOLUCIÓN PARA DEPLOY: ESCALAMIENTO A TODAS LAS IMÁGENES
✅ Cargado: /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_ok.csv
   Filas: 200

📊 ANÁLISIS COMPLETO:
- Total de registros en datos completos: 200
- Imágenes únicas requeridas: 200
- Rango de números de imágenes: 1 a 200
- Total de números únicos: 200

🔍 ESTRATEGIAS PARA DEPLOY:
1. 📱 USAR IMÁGENES PLACEHOLDER para números faltantes
2. 🔄 GENERAR URLs automáticamente basadas en patrón
3. ☁️  USAR CDN público para imágenes faltantes

🤖 GENERANDO MAPEO EXTENDIDO...
✅ URLs reales de Google Drive: 50
🔄 URLs placeholder para deploy: 150

🎯 APLICANDO MAPEO UNIVERSAL PARA DEPLOY...

🚀 CREANDO CSV FINAL PARA DEPLOY...
📊 RESULTADO FINAL PARA DEPLOY:
✅ Total de imágenes: 200
🌟 URLs reales de Google Drive: 50
🔄 URLs placeholder (funcionales): 150
📈 Cobertura real: 25.0%

💾 CSV para deploy guardado: /Users/n.arcos89/Documents/GitHub/Diva_digital/Data/publicaciones_pixabay_urls_ejemplo.csv

🎉 LISTO